### Importamos todo lo necesario

In [57]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os


### Variables de entorno


In [58]:
load_dotenv()
api_key = os.getenv("API_KEY")  

### Subimos el documento

In [59]:

def upload_pdf(url: str):        
    try:
        loader = PyPDFLoader(url)
        loader = loader.lazy_load()

        text = ""

        for page in loader: 
            text += page.page_content + "\n"

        return text
    except Exception as e:
        print(e)
        return []
    


## Text Splitter para separar todo el contenido de mi documento
### aumento del chunk size para la obtención de oraciones más largas
### Decremento del chunk_overlap para intentar hacerle perder el contexto al modelo, diviendo las frases importantes para que pierda la relación entre las palabras

- CharacterTextSplitter es más directo, divide el texto en trozos pequeños fijos.
- Aquí utilicé RecursiveCharacterTextSplitter que según la documentación es más inteligente y jerarquico, ideal para PDFs, libros o documentación

In [60]:

def text_splitter(text): 

    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 30
    )
    texts = text_splitter.create_documents([text])
    print(texts)
    return texts



# Defino el modelo que utilizaré

* Nuevo modelo de embedding de ollama (mxbai-embed-large): Es un modelo de incrustación (embedding) de lenguaje de última generación, de código abierto, desarrollado por Mixedbread.ai. 
* Su función principal es transformar texto (palabras, frases o documentos) en representaciones numéricas de alta dimensión, conocidas como vectores. Estos vectores capturan el significado semántico y sintáctico del texto, lo que permite a los sistemas de IA comprender la relación y similitud entre diferentes fragmentos de texto. 

* Este modelo solicitó otra vector Store ya que el anterior tenia un soporte para 768 dimensiones y este tiene hasta 1024 dimensiones al tratarse de un modelo de embedding más grande. Entonces, ¿a qué me refiero con dimensiones? la cantidad de números con los que cuenta un vector creado por el embedding

In [61]:
embedding = OllamaEmbeddings(
    model = "mxbai-embed-large:latest"
)

### Creo mi base de datos vectorial donde se guardará mi embedding

In [62]:
def get_vector_store(name_collection: str): 
    
    vector_store = Chroma(
    collection_name= name_collection,
    embedding_function=embedding,
    persist_directory="./prueba_chroma"
)    
    return vector_store

### Creo el retrieval que devolverá la información en una busqueda de similitudes

In [63]:
def retrieval(input_user: str): 
    vector_store = get_vector_store("langchainPrueba")
    docs = vector_store.similarity_search(input_user)
    return docs

### Creamos el propt system para el modelo

In [64]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre Arquitectura de software y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "perdon pero la pregunta excede mi conocimiento" y si te preguntan algo fuera del contexto principal responde "No estoy programado para eso".
    Utiliza siempre el contexto proporcionado para responder y también utiliza un lenguaje familiar y amigable, con carisma.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Creamos la función de respuesta que nos comunicará con nuestro agende de IA

In [65]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.0-flash",
    temperature= 0.7
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

### Utilizamos las funciones para cargar el doc, aplicarle el text_splitter y guardar esos datos como embedding en la base de datos vectorial

In [66]:
loader = upload_pdf("mi_tp.pdf")
texts = text_splitter(loader)
vector_store = get_vector_store("langchainPrueba")

vector_store.add_documents(texts)

[Document(metadata={}, page_content='1. Investiga y explica con tus palabras cada uno de los siguientes conceptos(podés usar libros, videos, artículos, papers o apuntes de clase):○ ¿Qué es una red neuronal artificial?Una red neuronal artificial es una red neuronal que es artificial porque no es real, es creada por nosotros, es decir es creada por el humano pero no de nacimiento sino que fue creado por nosotros (Son nodos que se encargan de realizar una predicción a partir de datos que se le brindan a través de tensores que pueden estar más o menos abstraidos dependiendo de la libreria que se utiliza)○ ¿Qué función cumplen las capas (entrada, ocultas, salida)?Las funciones que cumoplen las capas son:Entrada: Reciben los datos que le mandamos (ej: pixeles de una imagen, números, etc).Oculta: Se encargan de hacer los cálculos y las transformaciones inmediatas.Salida: Devuelve la predicción o un mensaje final de los datos recibidos, como por ej: “gato”, “spam”, “positivo”, etc.○ ¿Qué es un

['50948e39-cfb1-4477-83d2-f1112b5fbff5',
 '1c3d0d32-7835-4ec4-9ade-e6dd2780d7ee',
 'f704494e-60ea-40ed-9a30-5e14c5b56421',
 '4b761b41-b66b-442c-a6ad-3cd8ba17a3ab',
 '422b0238-4029-4da6-9519-14c1149f4873',
 '1b354047-4586-40fc-b04c-d6f7b23de235',
 '2bfd4ca4-4590-4d95-ae65-59cc4bc8b63a']

### Ponemos a prueba nuestro RAG
ya que se disminuyo el chunck_size y el overlap nos damos cuenta luego de la primera respuesta ya pierde el contexto.

In [67]:

for l in range(3):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end="", flush=True)
    

que es una neurona
[Document(id='b6d12b4c-d6a0-49b2-8ddd-4c1f8999ffe7', metadata={}, page_content='es el primer paso fundamental en el ciclo de entrenamiento de una red neuronal, después de que la'), Document(id='55a9dda8-e602-48e7-841a-13ba96e813f7', metadata={}, page_content='es el primer paso fundamental en el ciclo de entrenamiento de una red neuronal, después de que la'), Document(id='16b08668-ea45-4121-9ac6-ca5ff2fb19c7', metadata={}, page_content='es una función matemática que va a decidir si una neurona está activa o no, es decir, cuanta'), Document(id='06fbd9ac-9785-4e5e-847a-7488cfa3c7aa', metadata={}, page_content='es una función matemática que va a decidir si una neurona está activa o no, es decir, cuanta')]
Perdón, pero la pregunta excede mi conocimiento.
que es un nodo
[Document(id='599904a8-d68c-48f8-a380-6afc06539377', metadata={}, page_content='que fue creado por nosotros (Son nodos que se encargan de realizar una predicción a partir de datos'), Document(id='912c2928-8

### Al cambiar el chunck_size y el overlap dio una respuesta más concisa, al disminuirlo daba respuestas malas, no tan significativas y comprensibles

In [68]:

for l in range(3):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end=" ", flush=True)
    


[Document(id='08c11b6a-d857-4af3-a582-691a8777a3a8', metadata={}, page_content='a\n \nlos\n \nFM\n \npara\n \nobtener\n \ninformación\n \nespecífica\n \nde\n \nla\n \norganización\n \no\n \ndel'), Document(id='94a68a20-4b03-4f71-ab83-f7761b67f7b8', metadata={}, page_content='a\n \nlos\n \nFM\n \npara\n \nobtener\n \ninformación\n \nespecífica\n \nde\n \nla\n \norganización\n \no\n \ndel'), Document(id='191b8de8-1fb1-4e32-bbb2-5498a3a7d49f', metadata={}, page_content='entrenamiento  originales  para'), Document(id='a6bca68b-7e2a-4663-9740-097e687b1f88', metadata={}, page_content='entrenamiento  originales  para')]
Per don pero la pregunta excede mi conocimiento
  Backpropagation
[Document(id='e2dd635a-3dc4-44dc-bd13-efa5bb40d2ef', metadata={}, page_content='se relaciona con el cálculo dederivadas?La Backpropagation es un algoritmo clave que le permite a'), Document(id='1e74fe99-7d89-4536-a4e5-6b29b4d49c60', metadata={}, page_content='se relaciona con el cálculo dederivadas?La Backpropa